In [19]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy



import pickle
from tqdm import tqdm
import glob

import statistics 

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import pandas as pd

import warnings

np.seterr(all='raise')
scipy.special.seterr(all='raise')

{'singular': 'raise',
 'underflow': 'raise',
 'overflow': 'raise',
 'slow': 'raise',
 'loss': 'raise',
 'no_result': 'raise',
 'domain': 'raise',
 'arg': 'raise',
 'other': 'raise'}

In [20]:
tec_h = pd.read_csv('cms_csv/Hospital_General_Information.csv')
fnames = tec_h.loc[(tec_h['City']=='PHILADELPHIA') & (tec_h['State'] == 'PA')]['Facility Name'].unique()
fids = tec_h.loc[(tec_h['City']=='PHILADELPHIA') & (tec_h['State'] == 'PA')]['Facility ID'].unique()
facs = np.stack((fids,fnames),axis=1)
fids = facs[:,0].astype(str)
fnames = facs[:,1].astype(str)

# tec_h = pd.read_csv('cms_csv/Healthcare_Associated_Infections-Hospital.csv')
# fnames = tec_h.loc[(tec_h['City']=='PHILADELPHIA') & (tec_h['State'] == 'PA')]['Facility Name'].unique()
# fids = tec_h.loc[(tec_h['City']=='PHILADELPHIA') & (tec_h['State'] == 'PA')]['Facility ID'].unique()
# facs = np.stack((fids,fnames),axis=1)
# facs = np.delete(facs, 3,0)
# fids = facs[:,0].astype(str)
# fnames = facs[:,1].astype(str)

In [21]:
infections = pd.read_csv('cms_csv/Healthcare_Associated_Infections-Hospital.csv')
hac_reductions = pd.read_csv('cms_csv/FY_2022_HAC_Reduction_Program_Hospital.csv')
tec_h = pd.read_csv('cms_csv/Timely_and_Effective_Care-Hospital.csv')
payment_value = pd.read_csv('cms_csv/Payment_and_Value_of_Care-Hospital.csv')
comp_deaths = pd.read_csv('cms_csv/Complications_and_Deaths-Hospital.csv')
outp_img = pd.read_csv('cms_csv/Outpatient_Imaging_Efficiency-Hospital.csv')


infections = infections.loc[infections['Facility ID'].isin(list(fids))]
hac_reductions = hac_reductions.loc[hac_reductions['Facility ID'].isin(list(fids))]
tec_h = tec_h.loc[tec_h['Facility ID'].isin(list(fids))]
payment_value = payment_value.loc[payment_value['Facility ID'].isin(list(fids))]
comp_deaths = comp_deaths.loc[comp_deaths['Facility ID'].isin(list(fids))]
outp_img = outp_img.loc[outp_img['Facility ID'].isin(list(fids))]



In [22]:
#CALCULATING INFECTIONS SCORES FOR HOSPITALS, FOR 6 STANDARD PROCEDURES: HERE SCORE INDICATES THE RATE OF INFECTION:
# SIR - THE STANDARDIZED INFECTION RATIO - COMPARES THE HOSPITAL INFECTION RATIO TO THE NATIONAL AVERAGE
# HIGH SIR INDICATES HIGH RATE OF INFECTION, LOW INDICATES LOW RATE OF INFECTION

HAI_1_CILOWER = infections.loc[infections['Measure ID']=='HAI_1_CILOWER'][['Facility ID', 'Score']]
HAI_1_CIUPPER = infections.loc[infections['Measure ID']=='HAI_1_CIUPPER'][['Facility ID', 'Score']]
HAI_1_DOPC = infections.loc[infections['Measure ID']=='HAI_1_DOPC'][['Facility ID', 'Score']]
HAI_1_ELIGCASES = infections.loc[infections['Measure ID']=='HAI_1_ELIGCASES'][['Facility ID', 'Score']]
HAI_1_NUMERATOR = infections.loc[infections['Measure ID']=='HAI_1_NUMERATOR'][['Facility ID', 'Score']]
HAI_6_SIR = infections.loc[infections['Measure ID']=='HAI_6_SIR'][['Facility ID', 'Score']]

infections_scores = pd.merge(HAI_1_CIUPPER, HAI_1_CILOWER, on=['Facility ID'])
infections_scores = pd.merge(infections_scores, HAI_1_DOPC, on=['Facility ID'])
infections_scores = pd.merge(infections_scores, HAI_1_ELIGCASES, on=['Facility ID'])
infections_scores = pd.merge(infections_scores, HAI_1_NUMERATOR, on=['Facility ID'])
infections_scores = pd.merge(infections_scores, HAI_6_SIR, on=['Facility ID'])

infections_scores.columns = ['Facility ID', 'HAI_1_CIUPPER', 'HAI_1_CILOWER', 'HAI_1_DOPC', 
                             'HAI_1_ELIGCASES', 'HAI_1_NUMERATOR', 'HAI_6_SIR']

In [23]:
#CALCULATING TIMELY AND EFFECTIVE CARE SCORES FOR HOSPITALS, FOR 16 STANDARD PROCEDURES: 

EDV = tec_h.loc[tec_h['Measure ID']=='EDV'][['Facility ID', 'Score']] #EMERGENCY DEPARTMENT VOLUME
IMM_3 = tec_h.loc[tec_h['Measure ID']=='IMM_3'][['Facility ID', 'Score']] #PERCENT STAFF FLU IMMUNIZED
OP_18b = tec_h.loc[tec_h['Measure ID']=='OP_18b'][['Facility ID', 'Score']] #??
OP_18c = tec_h.loc[tec_h['Measure ID']=='OP_18c'][['Facility ID', 'Score']] #WAITING TIME

OP_2 = tec_h.loc[tec_h['Measure ID']=='OP_2'][['Facility ID', 'Score']] #
OP_22 = tec_h.loc[tec_h['Measure ID']=='OP_22'][['Facility ID', 'Score']]
OP_23 = tec_h.loc[tec_h['Measure ID']=='OP_23'][['Facility ID', 'Score']]
OP_29 = tec_h.loc[tec_h['Measure ID']=='OP_29'][['Facility ID', 'Score']]

OP_31 = tec_h.loc[tec_h['Measure ID']=='OP_31'][['Facility ID', 'Score']]
OP_3b = tec_h.loc[tec_h['Measure ID']=='OP_3b'][['Facility ID', 'Score']]
PC_01 = tec_h.loc[tec_h['Measure ID']=='PC_01'][['Facility ID', 'Score']]
SEP_1 = tec_h.loc[tec_h['Measure ID']=='SEP_1'][['Facility ID', 'Score']]

SEP_SH_3HR = tec_h.loc[tec_h['Measure ID']=='SEP_SH_3HR'][['Facility ID', 'Score']]
SEP_SH_6HR = tec_h.loc[tec_h['Measure ID']=='SEP_SH_6HR'][['Facility ID', 'Score']]
SEV_SEP_3HR = tec_h.loc[tec_h['Measure ID']=='SEV_SEP_3HR'][['Facility ID', 'Score']]
SEV_SEP_6HR = tec_h.loc[tec_h['Measure ID']=='SEV_SEP_6HR'][['Facility ID', 'Score']]

timely_effective_care_scores = pd.merge(EDV, IMM_3, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_18b, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_18c, on=['Facility ID'])

timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_2, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_22, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_23, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_29, on=['Facility ID'])

timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_31, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, OP_3b, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, PC_01, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, SEP_1, on=['Facility ID'])

timely_effective_care_scores = pd.merge(timely_effective_care_scores, SEP_SH_3HR, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, SEP_SH_6HR, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, SEV_SEP_3HR, on=['Facility ID'])
timely_effective_care_scores = pd.merge(timely_effective_care_scores, SEV_SEP_6HR, on=['Facility ID'])

timely_effective_care_scores.columns = ['Facility ID', 
                                        'EDV', 'IMM_3', 'OP_18b', 'OP_18c', 
                                        'OP_2', 'OP_22', 'OP_23', 'OP_29',
                                        'OP_31', 'OP_3b', 'PC_01', 'SEP_1',
                                        'SEP_SH_3HR', 'SEP_SH_6HR', 'SEV_SEP_3HR', 'SEV_SEP_6HR',]

In [24]:
# CALCULATING PAYMENT SCORES FOR HOSPITALS, FOR 4 STANDARD PROCEDURES: PAYMENT BY MEDICARE FOR A PROCEDURE:

PAYM_30_AMI = payment_value.loc[payment_value['Payment Measure ID']=='PAYM_30_AMI'][['Facility ID', 'Payment']]
PAYM_30_HF = payment_value.loc[payment_value['Payment Measure ID']=='PAYM_30_HF'][['Facility ID', 'Payment']]
PAYM_30_PN = payment_value.loc[payment_value['Payment Measure ID']=='PAYM_30_PN'][['Facility ID', 'Payment']]
PAYM_90_HIP_KNEE = payment_value.loc[payment_value['Payment Measure ID']=='PAYM_90_HIP_KNEE'][['Facility ID', 'Payment']]

payment_value_scores = pd.merge(PAYM_30_AMI, PAYM_30_HF, on=['Facility ID'])
payment_value_scores = pd.merge(payment_value_scores, PAYM_30_PN, on=['Facility ID'])
payment_value_scores = pd.merge(payment_value_scores, PAYM_90_HIP_KNEE, on=['Facility ID'])

payment_value_scores.columns = ['Facility ID', 'PAYM_30_AMI', 'PAYM_30_HF', 'PAYM_30_PN', 'PAYM_90_HIP_KNEE']

In [25]:
# CALCULATING COMPLICATION AND DEATH SCORES FOR HOSPITALS, FOR 10 STANDARD PROCEDURES: SCORE FOR A PROCEDURE:


COMP_HIP_KNEE = comp_deaths.loc[comp_deaths['Measure ID']=='COMP_HIP_KNEE'][['Facility ID', 'Score']]
MORT_30_AMI = comp_deaths.loc[comp_deaths['Measure ID']=='MORT_30_AMI'][['Facility ID', 'Score']]
MORT_30_CABG = comp_deaths.loc[comp_deaths['Measure ID']=='MORT_30_CABG'][['Facility ID', 'Score']]
MORT_30_COPD = comp_deaths.loc[comp_deaths['Measure ID']=='MORT_30_COPD'][['Facility ID', 'Score']]

MORT_30_HF = comp_deaths.loc[comp_deaths['Measure ID']=='MORT_30_HF'][['Facility ID', 'Score']]
MORT_30_PN = comp_deaths.loc[comp_deaths['Measure ID']=='MORT_30_PN'][['Facility ID', 'Score']]
MORT_30_STK = comp_deaths.loc[comp_deaths['Measure ID']=='MORT_30_STK'][['Facility ID', 'Score']]
PSI_03 = comp_deaths.loc[comp_deaths['Measure ID']=='PSI_03'][['Facility ID', 'Score']]

PSI_04 = comp_deaths.loc[comp_deaths['Measure ID']=='PSI_04'][['Facility ID', 'Score']]
PSI_06 = comp_deaths.loc[comp_deaths['Measure ID']=='PSI_06'][['Facility ID', 'Score']]


complication_death_score = pd.merge(COMP_HIP_KNEE, MORT_30_AMI, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, MORT_30_CABG, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, MORT_30_COPD, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, MORT_30_HF, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, MORT_30_PN, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, MORT_30_STK, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, PSI_03, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, PSI_04, on=['Facility ID'])
complication_death_score = pd.merge(complication_death_score, PSI_06, on=['Facility ID'])

complication_death_score.columns = ['Facility ID', 'COMP_HIP_KNEE', 'MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD',
                                   'MORT_30_HF', 'MORT_30_PN', 'MORT_30_STK', 'PSI_03', 'PSI_04', 'PSI_06',]

In [26]:
# CALCULATING OUTPATIENT IMAGING RATES FOR 3 STANDARD PROCEDURES: SCORE FOR A PROCEDURE:


OP_10 = outp_img.loc[outp_img['Measure ID']=='OP-10'][['Facility ID', 'Score']]
OP_13 = outp_img.loc[outp_img['Measure ID']=='OP-13'][['Facility ID', 'Score']]
OP_8 = outp_img.loc[outp_img['Measure ID']=='OP-8'][['Facility ID', 'Score']]

outp_img_score = pd.merge(OP_10, OP_13, on=['Facility ID'])
outp_img_score = pd.merge(outp_img_score, OP_8, on=['Facility ID'])

outp_img_score.columns = ['Facility ID', 'OP-10', 'OP-13', 'OP-8',]

In [27]:
# payment_value_scores
# timely_effective_care_scores
# infections_scores
# complication_death_score
# outp_img_score

# MERGE ALL INTO A COMPREHENSIVE DATAFRAME

comprehensive = pd.merge(payment_value_scores, timely_effective_care_scores.astype({'Facility ID': 'int64'}), on=['Facility ID'])
comprehensive = pd.merge(comprehensive, infections_scores.astype({'Facility ID': 'int64'}), on=['Facility ID'])
comprehensive = pd.merge(comprehensive, complication_death_score.astype({'Facility ID': 'int64'}), on=['Facility ID'])
comprehensive = pd.merge(comprehensive, outp_img_score.astype({'Facility ID': 'int64'}), on=['Facility ID'])

namesdf = pd.DataFrame(facs)
namesdf.columns = ['Facility ID', 'Facility Name']
namesdf = namesdf.astype({'Facility ID': 'int64'})

comprehensive = pd.merge(comprehensive, namesdf,  on=['Facility ID'])
comprehensive

,Facility ID,PAYM_30_AMI,PAYM_30_HF,PAYM_30_PN,PAYM_90_HIP_KNEE,EDV,IMM_3,OP_18b,OP_18c,OP_2,...,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_03,PSI_04,PSI_06,OP-10,OP-13,OP-8,Facility Name
0,390025,Not Available,Not Available,Not Available,Not Available,Not Available,97,Not Available,Not Available,Not Available,...,Not Available,Not Available,Not Available,0.59,Not Available,0.23,Not Available,Not Available,Not Available,KENSINGTON HOSPITAL
1,390026,"$25,114","$18,735","$21,291","$23,087",medium,93,157,Not Available,Not Available,...,10.5,16.2,11,1.25,152.28,0.29,4.6,Not Available,Not Available,CHESTNUT HILL HOSPITAL
2,390027,"$28,225","$17,997","$20,748","$23,070",very high,95,173,159,Not Available,...,9.5,13.3,15.4,0.47,148.01,0.31,5.1,6.5,Not Available,TEMPLE UNIVERSITY HOSPITAL
3,390111,"$27,959","$20,172","$19,881",Not Available,high,94,254,298,Not Available,...,8.7,12.6,14.1,0.41,164.22,0.20,2,5,43.2,HOSPITAL OF UNIV OF PENNSYLVANIA
4,390115,"$28,192","$17,779","$19,521","$21,811",very high,84,314,544,Not Available,...,9.5,15.1,13.4,0.10,148.52,0.19,1,Not Available,45.2,JEFFERSON HEALTH- NORTHEAST
5,390142,"$25,827","$18,623","$19,056","$21,822",very high,98,201,135,Not Available,...,8.6,15.1,14.5,1.12,144.19,0.26,2.3,3.2,Not Available,ALBERT EINSTEIN MEDICAL CENTER
6,390174,"$25,488","$17,522","$18,647","$18,519",very high,93,204,Not Available,Not Available,...,9.5,11.8,14.2,0.79,173.23,0.14,3.1,3.4,37.5,THOMAS JEFFERSON UNIVERSITY HOSPITAL
7,390204,"$27,743","$17,610","$18,882",Not Available,high,83,194,356,Not Available,...,13,17.4,14,2.17,Not Available,0.24,0.7,Not Available,Not Available,NAZARETH HOSPITAL
8,390223,"$25,597","$17,083","$18,223","$21,089",high,94,238,304,Not Available,...,8.2,13.5,12.4,0.39,150.61,0.25,1,4.4,Not Available,PENN PRESBYTERIAN MEDICAL CENTER
9,390226,"$25,640","$17,366","$18,847","$21,530",medium,Not Available,213,264,Not Available,...,10.1,13.8,10.8,0.14,148.18,0.23,2.2,5.6,Not Available,PENNSYLVANIA HOSPITAL


In [28]:
hospital_coords = [(39.981797172894, -75.13467141555005),
 (40.07967429866297, -75.21215067315889),
 (40.00780584468721, -75.15117725649382),
 (39.950271050746935, -75.19390435967095),
 (40.07119018805269, -74.98330939769801),
 (40.03678253762104, -75.1423426731601),
 (39.948111179580806, -75.15661832898245),
 (40.05941855830433, -75.04113154432343),
 (39.95843590736488, -75.19906475781814),
 (39.94598318876595, -75.15562180379865),
 (40.02913599171409, -75.20936414432435),
 (40.01941559041431, -75.09864320014417),
 (39.94812107974183, -75.15502878903119),
 (39.94890730278299, -75.19390254432663),
 (40.00600573651734, -75.12692077581774),
 (40.0073819087358, -75.1513754750135),]
 
 
 
 
 
 
 
 
 
 

In [29]:
comprehensive['coord'] = hospital_coords
comprehensive['lat'] = [i[0] for i in hospital_coords]
comprehensive['lon'] = [i[1] for i in hospital_coords]
# comprehensive

In [30]:
comprehensive

,Facility ID,PAYM_30_AMI,PAYM_30_HF,PAYM_30_PN,PAYM_90_HIP_KNEE,EDV,IMM_3,OP_18b,OP_18c,OP_2,...,PSI_03,PSI_04,PSI_06,OP-10,OP-13,OP-8,Facility Name,coord,lat,lon
0,390025,Not Available,Not Available,Not Available,Not Available,Not Available,97,Not Available,Not Available,Not Available,...,0.59,Not Available,0.23,Not Available,Not Available,Not Available,KENSINGTON HOSPITAL,"(39.981797172894, -75.13467141555005)",39.981797,-75.134671
1,390026,"$25,114","$18,735","$21,291","$23,087",medium,93,157,Not Available,Not Available,...,1.25,152.28,0.29,4.6,Not Available,Not Available,CHESTNUT HILL HOSPITAL,"(40.07967429866297, -75.21215067315889)",40.079674,-75.212151
2,390027,"$28,225","$17,997","$20,748","$23,070",very high,95,173,159,Not Available,...,0.47,148.01,0.31,5.1,6.5,Not Available,TEMPLE UNIVERSITY HOSPITAL,"(40.00780584468721, -75.15117725649382)",40.007806,-75.151177
3,390111,"$27,959","$20,172","$19,881",Not Available,high,94,254,298,Not Available,...,0.41,164.22,0.20,2,5,43.2,HOSPITAL OF UNIV OF PENNSYLVANIA,"(39.950271050746935, -75.19390435967095)",39.950271,-75.193904
4,390115,"$28,192","$17,779","$19,521","$21,811",very high,84,314,544,Not Available,...,0.10,148.52,0.19,1,Not Available,45.2,JEFFERSON HEALTH- NORTHEAST,"(40.07119018805269, -74.98330939769801)",40.071190,-74.983309
5,390142,"$25,827","$18,623","$19,056","$21,822",very high,98,201,135,Not Available,...,1.12,144.19,0.26,2.3,3.2,Not Available,ALBERT EINSTEIN MEDICAL CENTER,"(40.03678253762104, -75.1423426731601)",40.036783,-75.142343
6,390174,"$25,488","$17,522","$18,647","$18,519",very high,93,204,Not Available,Not Available,...,0.79,173.23,0.14,3.1,3.4,37.5,THOMAS JEFFERSON UNIVERSITY HOSPITAL,"(39.948111179580806, -75.15661832898245)",39.948111,-75.156618
7,390204,"$27,743","$17,610","$18,882",Not Available,high,83,194,356,Not Available,...,2.17,Not Available,0.24,0.7,Not Available,Not Available,NAZARETH HOSPITAL,"(40.05941855830433, -75.04113154432343)",40.059419,-75.041132
8,390223,"$25,597","$17,083","$18,223","$21,089",high,94,238,304,Not Available,...,0.39,150.61,0.25,1,4.4,Not Available,PENN PRESBYTERIAN MEDICAL CENTER,"(39.95843590736488, -75.19906475781814)",39.958436,-75.199065
9,390226,"$25,640","$17,366","$18,847","$21,530",medium,Not Available,213,264,Not Available,...,0.14,148.18,0.23,2.2,5.6,Not Available,PENNSYLVANIA HOSPITAL,"(39.94598318876595, -75.15562180379865)",39.945983,-75.155622


In [31]:
revenue = [10573700 , 994810075, 9271747070, 13262261573, 2098153996, 
           2949717829, 6050296835, 686953995, 4143789502, 3011575504, 
           312053341, 0, 108903535, 6139571764, 1492745083, 
          0, ]
total_discharges = [174, 6767, 28750, 36309, 24577, 
                    22718, 22718, 6458, 16213, 21941, 
                    2646, 375, 321, 17635,  3925, 
                   1, ]
revenue_per_discharge = list(np.array(revenue)//np.array(total_discharges))

In [32]:
condensed = pd.DataFrame()

#Average Medicare Payment
PAYM_30_AMI = []
for i in comprehensive['PAYM_30_AMI']:
    if i == 'Not Available':
        PAYM_30_AMI.append(float("nan"))
    else:
        PAYM_30_AMI.append(int(i.replace('$','').replace(',','')))
        
PAYM_30_HF = []
for i in comprehensive['PAYM_30_HF']:
    if i == 'Not Available':
        PAYM_30_HF.append(float("nan"))
    else:
        PAYM_30_HF.append(int(i.replace('$','').replace(',','')))
        
PAYM_30_PN = []
for i in comprehensive['PAYM_30_PN']:
    if i == 'Not Available':
        PAYM_30_PN.append(float("nan"))
    else:
        PAYM_30_PN.append(int(i.replace('$','').replace(',','')))
        
payment = [(PAYM_30_AMI[i]+PAYM_30_HF[i]+PAYM_30_PN[i])//3 if type(PAYM_30_AMI[i])==int else float("nan") for i in range(len(PAYM_30_AMI))]

#Average Infection Scores
# 'HAI_1_CIUPPER', 'HAI_1_CILOWER', 'HAI_1_DOPC', 'HAI_1_ELIGCASES', 'HAI_1_NUMERATOR', 'HAI_6_SIR'
# HAI_1_CIUPPER = []
# for i in comprehensive['HAI_1_CIUPPER']:
#     if i == 'Not Available':
#         HAI_1_CIUPPER.append(float("nan"))
#     else:
#         HAI_1_CIUPPER.append(i) 
# HAI_1_CILOWER = []
# for i in comprehensive['HAI_1_CILOWER']:
#     if i == 'Not Available':
#         HAI_1_CILOWER.append(float("nan"))
#     else:
#         HAI_1_CILOWER.append(i)   
# HAI_1_DOPC = []
# for i in comprehensive['HAI_1_DOPC']:
#     if i == 'Not Available':
#         HAI_1_DOPC.append(float("nan"))
#     else:
#         HAI_1_DOPC.append(i)
# HAI_1_ELIGCASES = []
# for i in comprehensive['HAI_1_ELIGCASES']:
#     if i == 'Not Available':
#         HAI_1_ELIGCASES.append(float("nan"))
#     else:
#         HAI_1_ELIGCASES.append(i)
# HAI_1_NUMERATOR = []
# for i in comprehensive['HAI_1_NUMERATOR']:
#     if i == 'Not Available':
#         HAI_1_NUMERATOR.append(float("nan"))
#     else:
#         HAI_1_NUMERATOR.append(i)
HAI_6_SIR = []
for i in comprehensive['HAI_6_SIR']:
    if i == 'Not Available':
        HAI_6_SIR.append(float("nan"))
    else:
        HAI_6_SIR.append(float(i))
        
#Timely and Effective Care: ED Waiting time
OP_18b = []
for i in comprehensive['OP_18b']:
    if i == 'Not Available':
        OP_18b.append(float("nan"))
    else:
        OP_18b.append(float(i))
OP_18c = []
for i in comprehensive['OP_18c']:
    if i == 'Not Available':
        OP_18c.append(float("nan"))
    else:
        OP_18c.append(float(i))
ed_wait = [(OP_18b[i]+OP_18c[i])//2 if (type(OP_18b[i])==float and type(OP_18c[i])==float) else float("nan") for i in range(len(OP_18c))]

#Outpatient imaging rates:
OP_10 = []
for i in comprehensive['OP-10']:
    if i == 'Not Available':
        OP_10.append(float("nan"))
    else:
        OP_10.append(float(i))
OP_13 = []
for i in comprehensive['OP-13']:
    if i == 'Not Available':
        OP_13.append(float("nan"))
    else:
        OP_13.append(float(i))
OP_8 = []
for i in comprehensive['OP-8']:
    if i == 'Not Available':
        OP_8.append(float("nan"))
    else:
        OP_8.append(float(i))
# op_imaging = [(OP_10[i]+OP_13[i]+OP_8[i])//3 if (type(OP_10[i])==float and type(OP_13[i])==float and (type(OP_8[i])==float)) 
#                                                                            else float("nan") for i in range(len(OP_18c))]
op_imaging = [OP_10[i] if type(OP_10[i])==float else float("nan") for i in range(len(OP_18c))]

MORT_30_AMI = []
for i in comprehensive['MORT_30_AMI']:
    if i == 'Not Available': MORT_30_AMI.append(float("nan"))
    else: MORT_30_AMI.append(float(i))
# MORT_30_CABG = []
# for i in comprehensive['MORT_30_CABG']:
#     if i == 'Not Available': MORT_30_CABG.append(float("nan"))
#     else: MORT_30_CABG.append(float(i))
MORT_30_COPD = []
for i in comprehensive['MORT_30_COPD']:
    if i == 'Not Available': MORT_30_COPD.append(float("nan"))
    else: MORT_30_COPD.append(float(i))
MORT_30_HF = []
for i in comprehensive['MORT_30_HF']:
    if i == 'Not Available': MORT_30_HF.append(float("nan"))
    else: MORT_30_HF.append(float(i))
MORT_30_PN = []
for i in comprehensive['MORT_30_PN']:
    if i == 'Not Available': MORT_30_PN.append(float("nan"))
    else: MORT_30_PN.append(float(i))
# MORT_30_STK = []
# for i in comprehensive['MORT_30_STK']:
#     if i == 'Not Available': MORT_30_STK.append(float("nan"))
#     else: MORT_30_STK.append(float(i))
mortality = [(MORT_30_AMI[i]+MORT_30_COPD[i]+MORT_30_HF[i]+MORT_30_PN[i])//4 
              if (type(MORT_30_AMI[i])==float and type(MORT_30_COPD[i])==float and type(MORT_30_HF[i])==float and type(MORT_30_PN[i])==float) 
                else float("nan") for i in range(len(MORT_30_AMI))]


condensed['Facility ID'] = comprehensive['Facility ID']
condensed['Facility Name'] = comprehensive['Facility Name']
condensed['lat'] = comprehensive['lat']
condensed['lon'] = comprehensive['lon']
condensed['avg_medicare_payment'] = payment
condensed['infection_ratio'] = HAI_6_SIR
condensed['emergency_wait_time'] = ed_wait
condensed['imaging_efficiency'] = op_imaging
condensed['mortality_30'] = mortality
condensed['revenue'] = revenue
condensed['total_discharges'] = total_discharges
condensed['revenue_per_discharge'] = revenue_per_discharge

In [33]:
# condensed
cols = list(condensed.columns)
cols.remove('Facility ID')
cols.remove('Facility Name')
cols.remove('lat')
cols.remove('lon')
for col in cols:
    col_zscore = col + '_zscore'
    condensed[col_zscore] = (condensed[col] - condensed[col].mean())/condensed[col].std(ddof=0)

In [34]:
condensed

,Facility ID,Facility Name,lat,lon,avg_medicare_payment,infection_ratio,emergency_wait_time,imaging_efficiency,mortality_30,revenue,total_discharges,revenue_per_discharge,avg_medicare_payment_zscore,infection_ratio_zscore,emergency_wait_time_zscore,imaging_efficiency_zscore,mortality_30_zscore,revenue_zscore,total_discharges_zscore,revenue_per_discharge_zscore
0,390025,KENSINGTON HOSPITAL,39.981797,-75.134671,NaN,NaN,NaN,NaN,NaN,10573700,174,60768,NaN,NaN,NaN,NaN,NaN,-0.849471,-1.125450,-1.020300
1,390026,CHESTNUT HILL HOSPITAL,40.079674,-75.212151,21713.0,0.665,NaN,4.6,11.0,994810075,6767,147009,0.408447,0.771498,NaN,1.619824,0.577350,-0.583859,-0.556708,-0.346566
2,390027,TEMPLE UNIVERSITY HOSPITAL,40.007806,-75.151177,22323.0,0.826,166.0,5.1,10.0,9271747070,28750,322495,1.200246,1.369190,-1.153067,1.948571,-0.288675,1.649808,1.339645,1.024371
3,390111,HOSPITAL OF UNIV OF PENNSYLVANIA,39.950271,-75.193904,22670.0,0.615,276.0,2.0,9.0,13262261573,36309,365260,1.650663,0.585880,0.190143,-0.089658,-1.154701,2.726713,1.991718,1.358461
4,390115,JEFFERSON HEALTH- NORTHEAST,40.071190,-74.983309,21830.0,0.454,429.0,1.0,10.0,2098153996,24577,85370,0.560317,-0.011812,2.058425,-0.747151,-0.288675,-0.286104,0.979663,-0.828104
5,390142,ALBERT EINSTEIN MEDICAL CENTER,40.036783,-75.142343,21168.0,0.604,168.0,2.3,11.0,2949717829,22718,129840,-0.298980,0.545044,-1.128645,0.107590,0.577350,-0.056295,0.819297,-0.480694
6,390174,THOMAS JEFFERSON UNIVERSITY HOSPITAL,39.948111,-75.156618,20552.0,0.775,NaN,3.1,9.0,6050296835,22718,266321,-1.098567,1.179859,NaN,0.633584,-1.154701,0.780447,0.819297,0.585527
7,390204,NAZARETH HOSPITAL,40.059419,-75.041132,21411.0,0.492,275.0,0.7,13.0,686953995,6458,106372,0.016442,0.129258,0.177932,-0.944399,2.309401,-0.666939,-0.583364,-0.664031
8,390223,PENN PRESBYTERIAN MEDICAL CENTER,39.958436,-75.199065,20301.0,0.332,271.0,1.0,10.0,4143789502,16213,255584,-1.424373,-0.464721,0.129088,-0.747151,-0.288675,0.265944,0.258147,0.501647
9,390226,PENNSYLVANIA HOSPITAL,39.945983,-75.155622,20617.0,0.266,238.0,2.2,10.0,3011575504,21941,137257,-1.014195,-0.709738,-0.273875,0.041840,-0.288675,-0.039602,0.752270,-0.422750


In [35]:
import json
result = condensed.to_json(path_or_buf='maps/Map/cms_data.json', orient='index')
result = condensed.to_json(path_or_buf='./cms_data.json', orient='index')